In [2]:
import os
import torch
import train
import ml4gw
import train.priors
import train.data.datasets.flow
import train.data.waveforms.generator.cbc
import ml4gw.waveforms 
import ml4gw.distributions
import ml4gw.transforms
import ml4gw.dataloading
from pathlib import Path

In [3]:
dec = ml4gw.distributions.Cosine()
psi = torch.distributions.Uniform(0, 3.14)
phi = torch.distributions.Uniform(-3.14, 3.14)
waveform_sampler = train.data.waveforms.generator.cbc.FrequencyDomainCBCGenerator(
    approximant=ml4gw.waveforms.IMRPhenomD(),
    f_min=20,
    f_max=1024,
    waveform_arguments={'f_ref': 40},
    num_test_waveforms=1,
    num_val_waveforms=1,
    parameter_sampler=train.priors.cbc_prior,
    num_fit_params=100000,
    inference_params=["chirp_mass", "mass_ratio", "distance", "phic",  "inclination", "dec", "psi", "phi"],
    sample_rate=2048,
    dec=dec,
    psi=psi,
    phi=phi, 
    duration=10)

datamodule = train.data.datasets.flow.FlowDataset(
    data_dir=Path(os.environ["AMPLFI_DATADIR"]),
    inference_params=["chirp_mass", "mass_ratio", "distance", "phic",  "inclination", "dec", "psi", "phi"],
    highpass=25,
    sample_rate=2048,
    kernel_length=3,
    fduration=1,
    psd_length=10,
    fftlength=2,
    batches_per_epoch=1,
    batch_size=1,
    ifos=["H1", "L1"],
    min_valid_duration=10000.0,
    waveform_sampler=waveform_sampler,
)

In [28]:
world_size, rank = datamodule.get_world_size_and_rank()
[background] = datamodule.load_background([datamodule.train_fnames[0]])
datamodule._logger = datamodule.get_logger(world_size, rank)
datamodule.build_transforms("fit")
dataset = ml4gw.dataloading.InMemoryDataset(
                background,
                kernel_size=int(datamodule.hparams.sample_rate * datamodule.sample_length),
                batch_size=datamodule.hparams.batch_size,
                coincident=False,
                batches_per_epoch=datamodule.hparams.batches_per_epoch,
                shuffle=True,
            )
dataloader = torch.utils.data.DataLoader(
            dataset, num_workers=2
        )
[X] = next(iter(dataloader))
X.to("cuda")

cross, plus, parameters = datamodule.waveform_sampler.sample(X)
strain, parameters = datamodule.inject(X, cross, plus, parameters)

2024-09-06 04:52:45,586 - AmplfiDataset - INFO - Building torch Modules and transferring to device
2024-09-06 04:52:45,591 - AmplfiDataset - INFO - Fitting standard scaler to parameters


RuntimeError: The size of tensor a (8192) must match the size of tensor b (402) at non-singleton dimension 2

In [29]:
# make tensors in x same length as cross tensor
[X] = X[:, :, :cross.shape[-1]]
strain, parameters = datamodule.inject(X, cross, plus, parameters)

RuntimeError: split_with_sizes expects split_sizes have only non-negative entries, but got split_sizes=[-7790, 8192]

In [ ]:
# cross, plus, parameters = datamodule.waveform_sampler.get_val_waveforms(rank, world_size)
# background = background[:][:len(cross[0])]
# datamodule.inject(background, cross, plus, parameters)